In [2]:
########################################################################
# import python-library
########################################################################
import pickle
import os
import sys
import glob

import numpy as np
import librosa
import librosa.core
import librosa.feature
import yaml
import logging
import keras

from tqdm import tqdm
from sklearn import metrics
from keras.models import Model
from keras.layers import Input, Dense
########################################################################



########################################################################
# setup STD I/O
########################################################################
# 로깅을 설정하고 초기화하는 부분
logging.basicConfig(level=logging.DEBUG, filename="make_pretrain_v1.log")
logger = logging.getLogger(' ')
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
########################################################################



########################################################################
# file I/O
########################################################################
#파일 입출력 관련 함수 선언 부분
def save_pickle(filename, save_data):  
    logger.info("save_pickle -> {}".format(filename))
    with open(filename, 'wb') as sf:
        pickle.dump(save_data, sf)

def load_pickle(filename):
    logger.info("load_pickle <- {}".format(filename))
    with open(filename, 'rb') as lf:
        load_data = pickle.load(lf)
    return load_data


def file_load(wav_name, mono=False):
    try:
        return librosa.load(wav_name, sr=None, mono=mono)
    except:
        logger.error("file_broken or not exists!! : {}".format(wav_name))


def demux_wav(wav_name, channel=0):
    try:
        multi_channel_data, sr = file_load(wav_name)
        if multi_channel_data.ndim <= 1:
            return sr, multi_channel_data
        return sr, np.array(multi_channel_data)[channel, :]
    except ValueError as msg:
        logger.warning(f'{msg}')
########################################################################



########################################################################
# feature extractor
########################################################################
#소리 파일 하나를 로그멜스펙트로그램 형태로 바꾸고 딥러닝 모델에 넣을 형태로 바꾸는 함수 
def file_to_vector_array(file_name, n_mels=64, frames=5, n_fft=1024, hop_length=512, power=2.0):
    dims = n_mels * frames
    sr, y = demux_wav(file_name)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, power=power)
    log_mel_spectrogram = 20.0 / power * np.log10(mel_spectrogram + sys.float_info.epsilon)
    vectorarray_size = len(log_mel_spectrogram[0, :]) - frames + 1
    if vectorarray_size < 1:
        return np.empty((0, dims), float)
    vectorarray = np.zeros((vectorarray_size, dims), float)
    for t in range(frames):
        vectorarray[:, n_mels * t: n_mels * (t + 1)] = log_mel_spectrogram[:, t: t + vectorarray_size].T 
    return vectorarray

#소리 파일들의 이름명이 담긴 리스트를 입력하면 그것들을 하나의 데이터셋으로 합치는 함수
def list_to_vector_array(file_list, msg="calc...", n_mels=64, frames=5, n_fft=1024, hop_length=512, power=2.0):
    dims = n_mels * frames
    for idx in tqdm(range(len(file_list)), desc=msg):
        vector_array = file_to_vector_array(file_list[idx], n_mels=n_mels, frames=frames, n_fft=n_fft, hop_length=hop_length,power=power)
        if idx == 0:
            dataset = np.zeros((vector_array.shape[0] * len(file_list), dims), float)
        dataset[vector_array.shape[0] * idx: vector_array.shape[0] * (idx + 1), :] = vector_array       
    return dataset

#기계 타입이 주어지면 id 전부의 normal 소리음들을 합쳐서 하나의 데이터셋으로 만드는 함수
def dataset_generator(target_dir, machine_type, normal_dir_name="normal", ext="wav"):
    logger.info("target_dir : {}".format(target_dir))
    train_files = []
    machine_id = f"-6dB_{machine_type}"
    machine = f"{machine_type}"
    id_list = ["id_00", "id_02", "id_04", "id_06"]
    machine_type_path = os.path.join(target_dir, machine_id, machine)
    for id_ in id_list:
        machine_id = f"{id_}"
        machine_id_path = os.path.join(machine_type_path, machine_id, normal_dir_name)
        normal_files = sorted(glob.glob(os.path.join(machine_id_path, f"*.{ext}")))
        train_files.extend(normal_files)  
    logger.info("train_file num : {num}".format(num=len(train_files)))
    return train_files
########################################################################




########################################################################
# keras model
########################################################################
#오토인코더 함수
def keras_model(inputDim):
    inputLayer = Input(shape=(inputDim,))
    h = Dense(64, activation="relu")(inputLayer)    
    h = Dense(64, activation="relu")(h)
    h = Dense(8, activation="relu")(h)
    h = Dense(64, activation="relu")(h)
    h = Dense(64, activation="relu")(h)
    h = Dense(inputDim, activation=None)(h)
    return Model(inputs=inputLayer, outputs=h)
########################################################################



########################################################################
# main
########################################################################
# 메인 실행 부분
if __name__ == "__main__":
    with open("make_pretrain_v1.yaml", encoding = 'utf-8') as stream:
        pretrain_v1_param = yaml.safe_load(stream)
    
    # pickle데이터, model데이터, result데이터가 저장될 파일과 폴더들 관련 변수 선언
    os.makedirs(pretrain_v1_param["pickle_directory"], exist_ok=True)
    os.makedirs(pretrain_v1_param["model_directory"], exist_ok=True)
    pretrain_v1_data_dir = pretrain_v1_param["base_directory"]

    machine_types = ["fan", "valve", "slider", "pump"]
    print("\n===========================")
    
    #각 기계 타입 별 normal 데이터 전체가 합쳐진 데이터를 생성하고 pretrain학습 시작 부분
    for machine_type in machine_types:
        db = f"-6dB_{machine_type}"
        train_pickle = "{pickle}/pretrain_only_{db}.pickle".format(pickle=pretrain_v1_param["pickle_directory"],db=db)
        
        #데이터셋 생성, pickle에 저장돼있다면 그것을 사용
        if os.path.exists(train_pickle):
            train_data = load_pickle(train_pickle)
        else:
            train_files = dataset_generator(pretrain_v1_data_dir, machine_type)
            train_data = list_to_vector_array(train_files,
                                              msg="generate train_dataset",
                                              n_mels=pretrain_v1_param["feature"]["n_mels"],
                                              frames=pretrain_v1_param["feature"]["frames"],
                                              n_fft=pretrain_v1_param["feature"]["n_fft"],
                                              hop_length=pretrain_v1_param["feature"]["hop_length"],
                                              power=pretrain_v1_param["feature"]["power"])
            save_pickle(train_pickle, train_data)

        print("============== MODEL TRAINING ==============")
        model_directory = pretrain_v1_param["model_directory"]
        model_file = "{model}/pretrain_only_{db}.h5".format(model=model_directory, db = db)
        if not os.path.exists(model_file):                                                                           
            #모델 생성, 학습, 및 저장
            model = keras_model(pretrain_v1_param["feature"]["n_mels"] * pretrain_v1_param["feature"]["frames"])
            model.summary()                    
            model.compile(**pretrain_v1_param["fit"]["compile"])
            model.fit(train_data,                 
                      train_data,
                      epochs=pretrain_v1_param["fit"]["epochs"],
                      batch_size=pretrain_v1_param["fit"]["batch_size"],
                      shuffle=pretrain_v1_param["fit"]["shuffle"],
                      validation_split=pretrain_v1_param["fit"]["validation_split"],
                      verbose=pretrain_v1_param["fit"]["verbose"])
            model.save(model_file)        
########################################################################

2023-08-25 23:57:56,372 - INFO - target_dir : ../Sample_data
2023-08-25 23:57:56,372 - INFO - target_dir : ../Sample_data
2023-08-25 23:57:56,378 - INFO - train_file num : 40
2023-08-25 23:57:56,378 - INFO - train_file num : 40


generate train_dataset: 100%|██████████████████████████████████████████████████████████| 40/40 [00:09<00:00,  4.05it/s]
2023-08-25 23:58:06,282 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_fan.pickle
2023-08-25 23:58:06,282 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_fan.pickle


============== MODEL TRAINING ==============
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 320)]             0         
                                                                 
 dense (Dense)               (None, 64)                20544     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)

C:\Users\Baejs\anaconda3\envs\bae\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2023-08-25 23:58:12,726 - INFO - target_dir : ../Sample_data
2023-08-25 23:58:12,726 - INFO - target_dir : ../Sample_data
2023-08-25 23:58:12,729 - INFO - train_file num : 40
2023-08-25 23:58:12,729 - INFO - train_file num : 40
generate train_dataset: 100%|██████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 68.21it/s]
2023-08-25 23:58:13,327 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_valve.pickle
2023-08-25 23:58:13,327 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_valve.pickle


============== MODEL TRAINING ==============
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320)]             0         
                                                                 
 dense_6 (Dense)             (None, 64)                20544     
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 8)                 520       
                                                                 
 dense_9 (Dense)             (None, 64)                576       
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Den

2023-08-25 23:58:19,768 - INFO - target_dir : ../Sample_data
2023-08-25 23:58:19,768 - INFO - target_dir : ../Sample_data
2023-08-25 23:58:19,771 - INFO - train_file num : 40
2023-08-25 23:58:19,771 - INFO - train_file num : 40
generate train_dataset: 100%|██████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 77.73it/s]
2023-08-25 23:58:20,293 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_slider.pickle
2023-08-25 23:58:20,293 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_slider.pickle


============== MODEL TRAINING ==============
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 320)]             0         
                                                                 
 dense_12 (Dense)            (None, 64)                20544     
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 8)                 520       
                                                                 
 dense_15 (Dense)            (None, 64)                576       
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Den

2023-08-25 23:58:26,160 - INFO - target_dir : ../Sample_data
2023-08-25 23:58:26,160 - INFO - target_dir : ../Sample_data
2023-08-25 23:58:26,163 - INFO - train_file num : 40
2023-08-25 23:58:26,163 - INFO - train_file num : 40
generate train_dataset: 100%|██████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 80.21it/s]
2023-08-25 23:58:26,670 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_pump.pickle
2023-08-25 23:58:26,670 - INFO - save_pickle -> ./pickle_pretrain_v1/pretrain_only_-6dB_pump.pickle


============== MODEL TRAINING ==============
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 320)]             0         
                                                                 
 dense_18 (Dense)            (None, 64)                20544     
                                                                 
 dense_19 (Dense)            (None, 64)                4160      
                                                                 
 dense_20 (Dense)            (None, 8)                 520       
                                                                 
 dense_21 (Dense)            (None, 64)                576       
                                                                 
 dense_22 (Dense)            (None, 64)                4160      
                                                                 
 dense_23 (Den